In [15]:
import pandas as pd

In [16]:
better_col_names = {
    'xgbClassifier': 'Simple Classification',
    'xgbOrdinalClassifier': 'Ordinal Classification',
    'xgbCoarseGrainedClassifier': 'Binned(3 Bins) Classification'
}

better_index_name = {
    'babelkw': 'Babelfy KWs',
    'kw': 'YAKE KWs',
    'wikikw': 'Wikifier KWs',
    'st': 'Sentence Transformer',
    'tfidf': 'TF-IDF'
}

In [17]:
results_dir = '../../ansible/results/accuracies'
parsed = dict()
with open(results_dir) as f:
    for l in f:
        label, acc = l.split('\t')
        parsed[label] = acc

In [26]:
results = []
for k in parsed:
    model, features, filtered = k.split('.')[0].split('_')
    filtered = filtered == 'True'
    accuracy = float(parsed[k].strip())
    results.append((model, features, filtered, accuracy))
df = pd.DataFrame(results, columns=['model', 'features', 'filtered', 'accuracy'])

In [27]:
df[df.filtered].\
    drop(columns=['filtered']).\
    pivot(index='model', columns='features').\
    transpose().\
    rename(columns=better_col_names, index=better_index_name).\
    style.highlight_max(color='lightgreen')

In [28]:
df[~df.filtered].\
    drop(columns=['filtered']).\
    pivot(index='model', columns='features').\
    transpose().\
    rename(columns=better_col_names, index=better_index_name).\
    style.highlight_max(color='lightgreen')